In [12]:
import getpass
import os
from dotenv import load_dotenv
from IPython.display import Markdown, display

# Load environment variables
load_dotenv()

if not os.environ.get("NVIDIA_API_KEY", "").startswith("nvapi-"):
    nvidia_api_key = getpass.getpass("Enter your NVIDIA API key: ")
    assert nvidia_api_key.startswith("nvapi-"), f"{nvidia_api_key[:5]}... is not a valid key"
    os.environ["NVIDIA_API_KEY"] = nvidia_api_key


In [13]:
from llama_index.core import Settings
from llama_index.llms.nvidia import NVIDIA

# Using the Phi-3.5-MOE-instruct model from the NVIDIA API Catalog
Settings.llm = NVIDIA(model="microsoft/phi-3.5-moe-instruct", api_key=os.getenv("NVIDIA_API_KEY"))


## Setting up the embedder 

In [14]:
from llama_index.embeddings.nvidia import NVIDIAEmbedding

Settings.embed_model = NVIDIAEmbedding(model="nvidia/nv-embedqa-e5-v5", api_key=os.getenv("NVIDIA_API_KEY"))


## Create an Azure AI Search Vector Store

In [15]:
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient, SearchIndexClient
from llama_index.vector_stores.azureaisearch import AzureAISearchVectorStore, IndexManagement

search_service_api_key = os.getenv('AZURE_SEARCH_ADMIN_KEY') or getpass.getpass('Enter your Azure Search API key: ')
search_service_endpoint = os.getenv('AZURE_SEARCH_SERVICE_ENDPOINT') or getpass.getpass('Enter your Azure Search service endpoint: ')
credential = AzureKeyCredential(search_service_api_key)

index_name = "llamaindex-nvidia-azureaisearch-demo"
index_client = SearchIndexClient(endpoint=search_service_endpoint, credential=credential)
search_client = SearchClient(endpoint=search_service_endpoint, index_name=index_name, credential=credential)

vector_store = AzureAISearchVectorStore(
    search_or_index_client=index_client,
    index_name=index_name,
    index_management=IndexManagement.CREATE_IF_NOT_EXISTS,
    id_field_key="id",
    chunk_field_key="chunk",
    embedding_field_key="embedding",
    embedding_dimensionality=1024,  # for nv-embedqa-e5-v5 model
)


ImportError: cannot import name 'SearchIndexClient' from 'azure.search.documents' (c:\Python312\Lib\site-packages\azure\search\documents\__init__.py)

 ## Load, Chunk, and Upload Documents

In [10]:
from llama_index.core import SimpleDirectoryReader, StorageContext, VectorStoreIndex
from llama_index.core.text_splitter import TokenTextSplitter

text_splitter = TokenTextSplitter(separator=" ", chunk_size=500, chunk_overlap=10)
documents = SimpleDirectoryReader(input_files=["Occupation Data.txt"]).load_data()
# storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex.from_documents(
    documents,
    transformations=[text_splitter],
    # storage_context=storage_context,
)

# Iterate through each document and print its chunked content
for i, doc in enumerate(documents):
    print(f"Document {i+1}:")
    for chunk in text_splitter.split_text(doc.text):
        print(chunk)
        print("-" * 50)


Document 1:
O*NET-SOC Code	Title	Description
11-1011.00	Chief Executives	Determine and formulate policies and provide overall direction of companies or private and public sector organizations within guidelines set up by a board of directors or similar governing body. Plan, direct, or coordinate operational activities at the highest level of management with the help of subordinate executives and staff managers.
11-1011.03	Chief Sustainability Officers	Communicate and coordinate with management, shareholders, customers, and employees to address sustainability issues. Enact or oversee a corporate sustainability strategy.
11-1021.00	General and Operations Managers	Plan, direct, or coordinate the operations of public or private sector organizations, overseeing multiple departments or locations. Duties and responsibilities include formulating policies, managing daily operations, and planning the use of materials and human resources, but are too diverse and general in nature to be classified 

## Create a Query Engine


In [21]:
query_engine = index.as_query_engine()
response = query_engine.query("What about a degree in Zoology? Quote your sources. ")
display(Markdown(f"{response}"))


 A degree in Zoology would fall under the category of Zoologists and Wildlife Biologists, who study the origins, behavior, diseases, genetics, and life processes of animals and wildlife. This information is sourced from the provided Occupation Data.txt file, specifically under the section for Zoologists and Wildlife Biologists (19-1023.00).

## Hybrid Retrieval Mode



In [19]:
# Initialize hybrid retriever and query engine
hybrid_retriever = index.as_retriever(vector_store_query_mode=VectorStoreQueryMode.HYBRID)
hybrid_query_engine = RetrieverQueryEngine(retriever=hybrid_retriever)

# Query execution
query = "What were the exact economic consequences mentioned in relation to Russia's stock market?"
response = hybrid_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))


NameError: name 'VectorStoreQueryMode' is not defined

## Hybrid + Semantic Ranking (Reranking) Retrieval Mode



In [ ]:
# Initialize hybrid retriever and query engine
semantic_reranker_retriever = index.as_retriever(vector_store_query_mode=VectorStoreQueryMode.SEMANTIC_HYBRID)
semantic_reranker_query_engine = RetrieverQueryEngine(retriever=semantic_reranker_retriever)

# Query execution
query = "What was the precise date when Russia invaded Ukraine?"
response = semantic_reranker_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))
